In [1]:
import pandas as pd
import numpy as np
import os
import torch

# Nhập đường dẫn cho 4 file dữ liệu của bạn
file_paths = {
    "1_1_1": r"D:\Study\5-FA25\AiTa_Lab_Research\Code\Bio_sequence_Research_AITALAB\train\task1_splicing_prediction\data_preparation\train_val\test_1_1_1.csv",
    "2_1_1": r"D:\Study\5-FA25\AiTa_Lab_Research\Code\Bio_sequence_Research_AITALAB\train\task1_splicing_prediction\data_preparation\train_val\test_2_1_1.csv",
    "4_1_1": r"D:\Study\5-FA25\AiTa_Lab_Research\Code\Bio_sequence_Research_AITALAB\train\task1_splicing_prediction\data_preparation\train_val\test_4_1_1.csv",
    "10_1_1": r"D:\Study\5-FA25\AiTa_Lab_Research\Code\Bio_sequence_Research_AITALAB\train\task1_splicing_prediction\data_preparation\train_val\test_10_1_1.csv",
    "100_1_1": r"D:\Study\5-FA25\AiTa_Lab_Research\Code\Inference_Model\test_data.csv"
}

output_dir = r"D:\Study\5-FA25\AiTa_Lab_Research\Code\Inference_Model\SpliceTransformer\data"
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

def validate_and_clean_seq(seq):
    """Lọc các ký tự không phải ACGT và đưa về chuẩn."""
    seq = str(seq).upper().strip()
    # Thay thế các ký tự lạ (N, R, Y,...) bằng 'N' 
    # Trong SpliceTransformer, 'N' thường được encode là [0.25, 0.25, 0.25, 0.25]
    allowed = set("ACGT")
    return "".join([c if c in allowed else "N" for c in seq])

def get_metadata(df):
    """Tính toán thông số để kiểm tra tính đồng nhất của dữ liệu."""
    lengths = df['sequence'].str.len().unique()
    return lengths

for ratio, path in file_paths.items():
    print(f"\n--- Đang xử lý file ratio: {ratio} ---")
    try:
        # 1. Load data
        df = pd.read_csv(path)
        
        # 2. Làm sạch trình tự
        df['sequence'] = df['sequence'].apply(validate_and_clean_seq)
        
        # 3. Tách Strand từ ID (ví dụ: Neg_21_46105846_+)
        df['strand'] = df['id'].apply(lambda x: x.split('_')[-1] if isinstance(x, str) else '+')
        
        # 4. Kiểm tra độ dài
        unique_lengths = get_metadata(df)
        print(f"Độ dài sequence trong file: {unique_lengths}")
        
        # 5. Lưu file sạch
        # Giữ lại các cột quan trọng nhất cho inference script
        output_df = df[['id', 'CHROM', 'strand', 'sequence', 'Splicing_types']]
        
        output_path = os.path.join(output_dir, f"prepared_inference_{ratio}.csv")
        output_df.to_csv(output_path, index=False)
        
        print(f"Thành công! File lưu tại: {output_path}")
        print(f"Số lượng mẫu: {len(output_df)}")
        
    except Exception as e:
        print(f"Lỗi tại file {ratio}: {e}")

print("\n--- TẤT CẢ ĐÃ SẴN SÀNG CHO INFERENCE ---")




--- Đang xử lý file ratio: 1_1_1 ---
Độ dài sequence trong file: [601]
Thành công! File lưu tại: D:\Study\5-FA25\AiTa_Lab_Research\Code\Inference_Model\SpliceTransformer\data\prepared_inference_1_1_1.csv
Số lượng mẫu: 26310

--- Đang xử lý file ratio: 2_1_1 ---
Độ dài sequence trong file: [601]
Thành công! File lưu tại: D:\Study\5-FA25\AiTa_Lab_Research\Code\Inference_Model\SpliceTransformer\data\prepared_inference_2_1_1.csv
Số lượng mẫu: 35132

--- Đang xử lý file ratio: 4_1_1 ---
Độ dài sequence trong file: [601]
Thành công! File lưu tại: D:\Study\5-FA25\AiTa_Lab_Research\Code\Inference_Model\SpliceTransformer\data\prepared_inference_4_1_1.csv
Số lượng mẫu: 52776

--- Đang xử lý file ratio: 10_1_1 ---
Độ dài sequence trong file: [601]
Thành công! File lưu tại: D:\Study\5-FA25\AiTa_Lab_Research\Code\Inference_Model\SpliceTransformer\data\prepared_inference_10_1_1.csv
Số lượng mẫu: 105708

--- Đang xử lý file ratio: 100_1_1 ---
Độ dài sequence trong file: [601]
Thành công! File lưu tạ

In [2]:
# Kiểm tra nhanh số lượng nhãn
print(df['Splicing_types'].value_counts())

# Kiểm tra xem có chuỗi nào chứa ký tự lạ không (ngoài ACGT)
import re
def check_unknown(seq):
    return bool(re.search(r'[^ACGT]', seq.upper()))

unknown_count = df['sequence'].apply(check_unknown).sum()
print(f"Số dòng chứa ký tự lạ: {unknown_count}")

Splicing_types
0    920809
1      8822
2      8666
Name: count, dtype: int64
Số dòng chứa ký tự lạ: 0
